<a href="https://colab.research.google.com/github/talhaanwarch/OffenseEval2020/blob/master/English/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
!pip install tensorflow==2.1.0


In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Embedding,Dense,Dropout,Flatten, Input
from tensorflow.keras.optimizers import RMSprop,Adam,Adadelta

# Load data

In [18]:
train=pd.read_csv( '/content/drive/My Drive/dataset/OLID/data/olid-training-v1.0.tsv',sep="\t")
train.head()

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN


In [19]:
val=pd.read_csv('/content/drive/My Drive/dataset/OLID/data/testset-levela.tsv',sep="\t")
val_labels=pd.read_csv('/content/drive/My Drive/dataset/OLID/data/labels-levela.csv',sep=",",header=None)
val_labels.columns=['idx','subtask_a']
#val_labels.head()
val=pd.concat([val,val_labels],axis=1)
val.head()

,id,tweet,idx,subtask_a
0,15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,15923,OFF
1,27014,"#ConstitutionDay is revered by Conservatives, ...",27014,NOT
2,30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,30530,NOT
3,13876,#Watching #Boomer getting the news that she is...,13876,NOT
4,60133,#NoPasaran: Unity demo to oppose the far-right...,60133,OFF


In [0]:
train_labels=train['subtask_a']
train=train['tweet']


val_labels=val['subtask_a']
val=val['tweet']


In [0]:
train=pd.concat([train,val])
labels=pd.concat([train_labels,val_labels])

In [22]:
test=pd.read_csv( '/content/drive/My Drive/dataset/OffenseEval2020/data/English/test_a_tweets.tsv',sep="\t")
test.head()

,id,tweet
0,A0,¿Who the fuck is Yoru?
1,A1,@USER @USER He's an evil law breaker that shou...
2,A2,Now hiring for 49 #Labor job opportunities in ...
3,A3,"#NerkondaPaarvai' - As Of Now, There Is NO Ear..."
4,A4,"@USER Hahahaha I wish...but a week is good, I’..."


In [23]:

train=train.str.replace('\d+', '')
train=train.str.replace('@USER', "") 
train=train.str.replace('URL', '')
train= train.str.lower()
#train = train.str.replace('[^\w\s]','')
#train = train.apply(lambda x : [lemmatizer.lemmatize(y) for y in w_tokenizer.tokenize(x)])
#train = train.apply(lambda x: [item for item in x if item not in stop])
#train = train.apply(lambda x : " ".join(x))
train.head()

0     she should ask a few native americans what th...
1          go home you’re drunk!!!  #maga #trump 👊🇺🇸👊 
2    amazon is investigating chinese employees who ...
3     someone should'vetaken" this piece of shit to...
4      obama wanted liberals &amp; illegals to move...
Name: tweet, dtype: object

In [24]:

test['tweet']=test['tweet'].str.replace('\d+', '')
test['tweet']=test['tweet'].str.replace('@USER', "") 
test['tweet']=test['tweet'].str.replace('URL', '')
test["tweet"]= test["tweet"].str.lower()
# test["tweet"] = test["tweet"].str.replace('[^\w\s]','')
# test["tweet"] = test["tweet"].apply(lambda x : [lemmatizer.lemmatize(y) for y in w_tokenizer.tokenize(x)])
# test["tweet"] = test["tweet"].apply(lambda x: [item for item in x if item not in stop])
# test["tweet"] = test["tweet"].apply(lambda x : " ".join(x))
test.head()

,id,tweet
0,A0,¿who the fuck is yoru?
1,A1,he's an evil law breaker that should be in p...
2,A2,now hiring for #labor job opportunities in #m...
3,A3,"#nerkondapaarvai' - as of now, there is no ear..."
4,A4,"hahahaha i wish...but a week is good, i’m ext..."


#Label Encoding

In [0]:
test=test['tweet']


In [26]:
train.head()

0     she should ask a few native americans what th...
1          go home you’re drunk!!!  #maga #trump 👊🇺🇸👊 
2    amazon is investigating chinese employees who ...
3     someone should'vetaken" this piece of shit to...
4      obama wanted liberals &amp; illegals to move...
Name: tweet, dtype: object

In [27]:
le=LabelEncoder()
labels=le.fit_transform(labels)
print(len(labels))

14100


In [0]:
train_bert=train.copy()
test_bert=test.copy()

#BERT

In [14]:
!pip install sentencepiece
!pip install tensorflow-hub

     |████████████████████████████████| 1.0MB 2.8MB/s 


In [0]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [0]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=2e-6), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [36]:
%%time
import tensorflow_hub as hub
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

CPU times: user 9.34 s, sys: 1.64 s, total: 11 s
Wall time: 10.8 s


In [0]:
!wget --quiet https://raw.githubusercontent.com/google-research/ALBERT/master/tokenization.py
import tokenization
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [40]:
len(train_bert),len(test_bert)

(14100, 3887)

In [41]:
train_bert.values[0:5]

array([' she should ask a few native americans what their take on this is.',
       '  go home you’re drunk!!!  #maga #trump 👊🇺🇸👊 ',
       'amazon is investigating chinese employees who are selling internal data to third-party sellers looking for an edge in the competitive marketplace.  #amazon #maga #kag #china #tcot',
       ' someone should\'vetaken" this piece of shit to a volcano. 😂"',
       '  obama wanted liberals &amp; illegals to move into red states'],
      dtype=object)

In [0]:
train_input = bert_encode(train_bert.values, tokenizer, max_len=160)
test_input = bert_encode(test_bert.values, tokenizer, max_len=160)
train_labels = labels

In [43]:
len(train_input),len(train_labels),len(test_input)

(3, 14100, 3)

In [44]:
bert_model = build_model(bert_layer, max_len=160)
bert_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 160)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 160)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 160)]        0                                            
__________________________________________________________________________________________________
keras_layer_2 (KerasLayer)      [(None, 1024), (None 335141889   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [0]:
train_history = bert_model.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=5,
    batch_size=16)


Train on 11280 samples, validate on 2820 samples
Epoch 1/5
11280/11280 [==============================] - 1265s 112ms/sample - loss: 0.5188 - accuracy: 0.7416 - val_loss: 0.4271 - val_accuracy: 0.8043
Epoch 2/5
 5120/11280 [============>.................] - ETA: 10:20 - loss: 0.4046 - accuracy: 0.8182